Serial Schedule Generation Scheme (SGS)

The solver iteratively schedules jobs that are ready (all predecessors completed). For each ready job, it determines the earliest feasible start time by considering:

* Precedence Constraints: Must start after all predecessors finish.
* Resource Constraints: Must start when sufficient resources are available for its entire duration.

The algorithm maintains a timeline to track resource usage over time and incrementally finds the first available time slot that satisfies all constraints.

Heuristics Employed:
* Job Prioritization: When multiple jobs are ready, the solver prioritizes jobs with more successors (ready_queue.sort(key=lambda j: len(adj[j]), reverse=True)). This aims to "unblock" the dependency graph faster.
* Earliest Feasible Start Time: The solver searches for the absolute earliest time a job can begin by incrementally checking time steps from its earliest possible start time until resource availability is confirmed.
* Mode Selection: If a job has multiple operational modes, the solver defaults to using the first mode listed in the input data (_get_job_data method).

In [1]:
import json
import time
from collections import defaultdict

class SchedulingSolver:
    def __init__(self, data):
        self.data = data
        self.jobs = data.get('jobs', [])
        self.resources = data.get('resources', [])
        
    def _get_job_data(self, job):
        """
        Extracts duration and resources from nested 'modes' list.
        Defaults to the first mode if multiple exist.
        """
        modes = job.get('modes', [])
        if not modes:
            return 0, {}
        
        # Taking the first mode as per your dataset structure
        first_mode = modes[0]
        duration = first_mode.get('duration', 0)
        resources = first_mode.get('resources_required', {})
        return duration, resources

    def solve(self):
        start_time_bench = time.time()
        
        # 1. Setup Resource Capacities
        capacities = {str(r['id']): r.get('capacity', 0) for r in self.resources}
        
        # 2. Build Precedence Graph
        job_lookup = {str(j['id']): j for j in self.jobs}
        adj = defaultdict(list)
        in_degree = {str(j['id']): 0 for j in self.jobs}
        
        for job in self.jobs:
            jid = str(job['id'])
            successors = job.get('successors', [])
            for succ in successors:
                succ_id = str(succ)
                if succ_id in job_lookup:
                    adj[jid].append(succ_id)
                    in_degree[succ_id] += 1

        # 3. Initialize Scheduling Variables
        ready_queue = [jid for jid, deg in in_degree.items() if deg == 0]
        schedule = {}      
        finish_times = {}  
        timeline = defaultdict(lambda: defaultdict(int))

        # 4. Serial Schedule Generation Scheme
        while ready_queue:
            # Priority: Most successors first to unblock the DAG
            ready_queue.sort(key=lambda j: len(adj[j]), reverse=True)
            
            curr_id = ready_queue.pop(0)
            job = job_lookup[curr_id]
            
            # Use the helper to get nested mode data
            duration, reqs = self._get_job_data(job)

            # Precedence Constraint: Max finish time of all predecessors
            predecessors = [p_id for p_id, succs in adj.items() if curr_id in succs]
            t_start = max([finish_times[p] for p in predecessors], default=0)

            # Resource Constraint: Find the first available window
            while True:
                is_feasible = True
                if duration > 0:
                    for t in range(t_start, t_start + duration):
                        for res_id, amount in reqs.items():
                            res_key = str(res_id)
                            if timeline[t][res_key] + amount > capacities.get(res_key, 0):
                                is_feasible = False
                                break
                        if not is_feasible: break
                
                if is_feasible:
                    schedule[curr_id] = t_start
                    finish_times[curr_id] = t_start + duration
                    
                    if duration > 0:
                        for t in range(t_start, t_start + duration):
                            for res_id, amount in reqs.items():
                                timeline[t][str(res_id)] += amount
                    break
                else:
                    t_start += 1 

            for succ in adj[curr_id]:
                in_degree[succ] -= 1
                if in_degree[succ] == 0:
                    ready_queue.append(succ)

        makespan = max(finish_times.values()) if finish_times else 0
        
        return {
            "schedule": schedule,
            "metrics": {
                "makespan": makespan,
                "computation_time_ms": round((time.time() - start_time_bench) * 1000, 6)
            }
        }

In [3]:
with open("C:/Users/Admin/Desktop/PSPLIB_j301_1.json", 'r') as f:
    dataset = json.load(f)  

solver = SchedulingSolver(dataset)
result = solver.solve()
print(json.dumps(result, indent=2))

{
  "schedule": {
    "1": 0,
    "2": 0,
    "3": 8,
    "4": 0,
    "8": 12,
    "11": 8,
    "13": 12,
    "10": 6,
    "19": 21,
    "18": 18,
    "16": 13,
    "6": 23,
    "15": 12,
    "7": 12,
    "5": 17,
    "20": 23,
    "9": 6,
    "12": 21,
    "26": 17,
    "29": 30,
    "21": 31,
    "27": 33,
    "25": 30,
    "14": 37,
    "28": 41,
    "17": 41,
    "31": 44,
    "22": 47,
    "23": 54,
    "24": 56,
    "30": 59,
    "32": 61
  },
  "metrics": {
    "makespan": 61,
    "computation_time_ms": 0.0
  }
}
